# 3ª Implementação do Sistema de Recomendação
Especificação: Context based<br>
1 - pegar a descrição do filme (sinopse) numa base (imdb)<br>
2 - pegar os top 10 filmes que o usuario assistiu e usar pra criar um texto sobre o usuário<br>
avaliar ranking: <br>
map (mean average precision) nDCG (normalized discounted cumulative gain) (pra todos os métodos)

## 1ª parte: levantamento dos dados
1. Carregar a base de ratings dos usuários por filmes do MovieLens (https://files.grouplens.org/datasets/movielens/ml-25m.zip 2019-12-03 11:14	250M)
2. Carregar a base de sinopses
    - Do **Kaggle**, obtido 8547 sinopses: movie_synopsis.csv = https://www.kaggle.com/datasets/linggarmaretva/movie-synopsis-dataset/versions/1?resource=download
    - Demais sinopses obtidas por requisição de **TMDB** (www.themoviedb.org)
3. Tratar os títulos dos filmes da base para que sejam compatíveis com de bases de ratings
    - Título no formato: *The/A/An Object*, no lugar de: *Object, The (a.k.a. Objeto, O)* 
4. Unificar a base para ir para próxima parte
    - Usar o *ano de produção* do filme junto com o *título* para identificar sua sinopse

In [2]:
import pandas as pd
import IPython.display as disp
import requests
import math
import re

movies = pd.read_csv('movies.csv')           #obtido de MovieLens
sinopses = pd.read_csv('movie_synopsis.csv') #obtido de Kaggle e parece ser mesma sinopse que temos em TMDB.org
disp.display(movies.head(2))

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [3]:
disp.display(sinopses.head(3))                      #Checar dados do Kaggle
movies['titulo'] = movies['title'].str[:-7]         #retirar a informação de ano do filme campo 'title' para uma nova coluna chamada 'titulo'
movies['ano'] = movies['title'].str.extract("\((\d{4})\)", expand=True)     #criar uma coluna ano com o ano de produção do mesmo.
disp.display(movies.head(3))                        #Trabalharemos com as colunas 'titulo' e 'ano'

,title,synopsis
0,Four Rooms,It's Ted the Bellhop's first night on the job....
1,Judgment Night,"While racing to a boxing match, Frank, Mike, J..."
2,Life in Loops (A Megacities RMX),Timo Novotny labels his new project an experim...


,movieId,title,genres,titulo,ano
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men,1995


In [12]:
print(f'Tenho a sinopse de {len(sinopses)} filmes')
print(f'Dos {len(movies)} filmes, tenho {len(set(movies.titulo)-set(sinopses.title))} sem sinopse')
n_duplicados = movies.duplicated(subset=['titulo']).value_counts()
print(f'{n_duplicados[False]} filmes com título único e {n_duplicados[True]} com nomes repetidos')
#print(*movies[movies.duplicated(subset=['titulo'])].sort_values('titulo')['titulo'].unique(), sep="; ") #listar todos os nomes que aparecem duplicados

Tenho a sinopse de 8457 filmes
Dos 62423 filmes, tenho 53199 sem sinopse
58513 filmes com título único e 3910 com nomes repetidos


In [9]:
movies[movies['titulo']=='Aladdin'] #Essa amostragem sugere que usar o nome e ano pode ser uma boa estratégia para identificar a sinopse correta de uma produção

,movieId,title,genres,titulo,ano
580,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical,Aladdin,1992
22286,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,Aladdin,1992
23734,119061,Aladdin (1986),Adventure|Fantasy,Aladdin,1986
59296,200540,Aladdin (2019),Adventure|Fantasy|Romance,Aladdin,2019


In [43]:
movies[movies['titulo']=='Misérables, Les']
#Idéia para implementatação futura: contar o generos dos filmes repetidos, e definir que o genero que acontece mais vezes é o prevalente, 
# logo todos os filmes com mesmo nome e mesmo gênero podem ser regravações e terem a mesmo sinopse

,movieId,title,genres,titulo,ano
72,73,"Misérables, Les (1995)",Drama|War,"Misérables, Les",1995
1784,1873,"Misérables, Les (1998)",Crime|Drama|Romance|War,"Misérables, Les",1998
13928,72117,"Misérables, Les (1935)",Drama|Romance,"Misérables, Les",1935
16981,89349,"Misérables, Les (1934)",Drama,"Misérables, Les",1934
19028,99149,"Misérables, Les (2012)",Drama|Musical|Romance|IMAX,"Misérables, Les",2012
19107,99560,"Misérables, Les (1958)",Drama,"Misérables, Les",1958
19108,99562,"Misérables, Les (1952)",Drama|Romance,"Misérables, Les",1952
20398,105509,"Misérables, Les (1978)",Drama,"Misérables, Les",1978


#### Tratar os títulos 
que terminam com artigo e passá-lo pro começo do título para comparar com demais bases

In [13]:
def mover_artigo_do_titulo(artigo):
    try:
        print(len(movies[movies['titulo'].str.endswith(', '+artigo)]),' filmes encontrados terminando com: \', ',artigo,'\'')
    except ValueError:
        print('Nenhum título de filme terminando com : ', artigo)
    movies.loc[movies['titulo'].str.endswith(', '+artigo), 'titulo'] = artigo +' '+movies[movies['titulo'].str.endswith(', '+artigo)]['titulo'].str[:-int(len(artigo)+2)]    
 #   movies.loc[movies['titulo'].str.contains(f',\s({artigo})\s\('), 'titulo'] = artigo +' '+ ###consertar os titulos aqui tb para começar a baixar via request mais json

artigos = ['A','As','O','Os','Le','Les','Lo','Los','Las','La','The','El','L\'', 'An']
for a in artigos:
    mover_artigo_do_titulo(a)

333  filmes encontrados terminando com: ',  A '
0  filmes encontrados terminando com: ',  As '
1  filmes encontrados terminando com: ',  O '
0  filmes encontrados terminando com: ',  Os '
12  filmes encontrados terminando com: ',  Le '
16  filmes encontrados terminando com: ',  Les '
0  filmes encontrados terminando com: ',  Lo '
3  filmes encontrados terminando com: ',  Los '
1  filmes encontrados terminando com: ',  Las '
26  filmes encontrados terminando com: ',  La '
3398  filmes encontrados terminando com: ',  The '
9  filmes encontrados terminando com: ',  El '


Fazer uma função genérica que trabalha só com String e não necessita percorrer o dataset como a anterior

In [137]:
def set_artigo(titulo):
    artigos = ['A','As','O','Os','Le','Les','Lo','Los','Las','La','The','El','L\'', 'An', 'Il'] #Identificados artigos em inglês, português, espanhol, italiano, francês
    for art in artigos:
        if(titulo.endswith(', ' + art)):
            return art+' '+titulo[:-int(len(art)+2)]
    return titulo

alguns títulos tem um segundo nome (tipo: A.K.A.: Also Know As) ou o nome em um idioma original entre parenteses. Estes nomes todos poderão ser usados para identificação do filme

In [190]:
def get_parenteses(titulo):
    resposta = []
    for subtitulo in re.findall(r'\([^()]*\)', titulo): # encontrar substrings dentro dos parenteses
        x = subtitulo.strip('()')
        if x.startswith('a.k.a. '):
            x = x[7:]
        elif x.startswith('aka '):
            x = x[4:]
        resposta.append(x)
    for subtitulo in re.split(r'\([^()]*\)', titulo): # encontrar substrings fora dos parenteses
        resposta.append(subtitulo.strip())

    for i in range(len(resposta)):
        if ', ' in resposta[i]:
            resposta[i]= set_artigo(resposta[i])
    return list(filter(lambda s: s != '', resposta)) #remover da lista final os resultados vazios caso os tenha

Testar as duas funções acima:

In [189]:
get_parenteses('Twelve Monkeys (a.k.a. 12 Monkeys)')

['12 Monkeys', 'Twelve Monkeys']

In [136]:
set_artigo('Cité des enfants perdus, La')

'La Cité des enfants perdus'

In [20]:
movies[movies.index.isin([205,236,255,259,291,27345,29855,33562,34085,38949, 37254,])]

,movieId,title,genres,titulo,ano
205,207,"Walk in the Clouds, A (1995)",Drama|Romance,A Walk in the Clouds,1995
236,239,"Goofy Movie, A (1995)",Animation|Children|Comedy|Romance,A Goofy Movie,1995
255,258,"Kid in King Arthur's Court, A (1995)",Adventure|Children|Comedy|Fantasy|Romance,A Kid in King Arthur's Court,1995
259,262,"Little Princess, A (1995)",Children|Drama,A Little Princess,1995
291,295,"Pyromaniac's Love Story, A (1995)",Comedy|Romance,A Pyromaniac's Love Story,1995
27345,128910,"One-Way Trip to Antibes, A (2011)",Drama,A One-Way Trip to Antibes,2011
29855,134863,"Big Love Story, A (2012)",Comedy|Romance,A Big Love Story,2012
33562,143444,"Bigger Splash, A (2015)",Comedy|Crime|Drama|Mystery,A Bigger Splash,2015
34085,144622,"Ballerina's Tale, A (2015)",Documentary,A Ballerina's Tale,2015
37254,152230,"First, the Last, The (Les premiers les dernier...",(no genres listed),"First, the Last, The (Les premiers les derniers)",2016


In [21]:
a=0
b=0
c=0
for index, linha  in sinopses.iterrows():   #para cada linha do arquivo de sinopses
    nome_filme = (linha['title'])
    if (len(movies[movies['titulo']==nome_filme])==1):
        movies.loc[movies['titulo']==nome_filme,'sinopse'] = linha['synopsis']
        a+=1
        movies[movies['titulo']==nome_filme]
    elif (len(movies[movies['titulo']==nome_filme])==0):
        b+=1
    elif (len(movies[movies['titulo']==nome_filme])>1):
        c+=1
print(f'Achou {a} compativeis, {b} que tem sinopse e não estão na base e {c} que tem mais de uma refência (duplicados)')
movies

Achou 5319 compativeis, 1921 que tem sinopse e não estão na base e 1217 que tem mais de uma refência (duplicados)


,movieId,title,genres,titulo,ano,sinopse
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,"Led by Woody, Andy's toys live happily in his ..."
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji,1995,When siblings Judy and Peter discover an encha...
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men,1995,A family wedding reignites the ancient feud be...
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale,1995,NaN
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II,1995,Just when George Banks has recovered from his ...
...,...,...,...,...,...,...
62418,209157,We (2018),Drama,We,2018,NaN
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul,2001,NaN
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems,2018,NaN
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing,2001,NaN


##### Premissas:
1. Filmes são identificados por Título e Ano de produção.
2. Filmes com mesmo nome feitos em anos distintos serão tratados como histórias diferentes sob a mesma titularidade, logo sumários diferentes 
2. Dois filmes do mesmo ano com mesmo nome devem ser o mesmo filme cadastrado duas vezes no banco de dados, deve ser falha no cadastro

In [238]:
import TMDB_key

def get_sumario(title = "Star Wars", year = 1977, prints=False):
    api_key = TMDB_key.api_key

    # Enviar uma solicitação GET à API do TMDb para pesquisar por filmes
    response = requests.get(f"https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={title}&year={year}")
    
    if response.status_code == 200:     # se a solicitação foi bem-sucedida        
        data = response.json() # Carregar o conteúdo da resposta em um dicionário Python
        try:        
            result = data["results"][0] # Obter o primeiro resultado da pesquisa
        except IndexError:
            return float('NAN')
        if(prints):
            print(f"Título: {result['title']}")
            print(f"Sumário: {result['overview']}")
    else:
        print("Ocorreu um erro ao pesquisar por filmes. Response:",response.status_code)
        return

    return result['overview']

In [229]:
def salvar_movies2():
    global movies2
    movies2.to_csv('movies2.csv', index=False)
    movies2 = pd.read_csv('movies2.csv')
    movies2.ano = movies2.ano.fillna(0).astype(int)
salvar_movies2()
movies2

,movieId,title,genres,titulo,ano,sinopse
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,"Led by Woody, Andy's toys live happily in his ..."
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji,1995,When siblings Judy and Peter discover an encha...
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men,1995,A family wedding reignites the ancient feud be...
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale,1995,"Cheated on, mistreated and stepped on, the wom..."
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II,1995,Just when George Banks has recovered from his ...
...,...,...,...,...,...,...
62418,209157,We (2018),Drama,We,2018,"Five years after meeting her three fathers, So..."
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul,2001,Nineteen people with differing degrees of visu...
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems,2018,"33-years old Tamás Merthner is heartbroken, af..."
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing,2001,A Girl Thing is a mini-series that revolves ar...


In [228]:
movies2[movies2['sinopse'].isnull()]

,movieId,title,genres,titulo,ano,sinopse
32,33,Wings of Courage (1995),Adventure|Romance|IMAX,Wings of Courage,1995,NaN
82,83,Once Upon a Time... When We Were Colored (1995),Drama|Romance,Once Upon a Time... When We Were Colored,1995,NaN
104,106,Nobody Loves Me (Keiner liebt mich) (1994),Comedy|Drama,Nobody Loves Me (Keiner liebt mich),1994,NaN
126,128,Jupiter's Wife (1994),Documentary,Jupiter's Wife,1994,NaN
181,183,Mute Witness (1994),Comedy|Horror|Thriller,Mute Witness,1994,NaN
...,...,...,...,...,...,...
62185,208199,Rufus (2013),Drama,Rufus,2013,NaN
62212,208279,"Mayflower, Quiltshoe & The Rubens Brothers (2017)",Children|Comedy,"Mayflower, Quiltshoe & The Rubens Brothers",2017,NaN
62305,208695,Vi på Saltkråkan (1968),Children,Vi på Saltkråkan,1968,NaN
62355,208859,Screen Play (1992),(no genres listed),Screen Play,1992,NaN


In [227]:
filmes_sem_sinopse = movies2[movies2['sinopse'].isnull()].sort_index(ascending=False).iterrows()
c=0
for index, filme in filmes_sem_sinopse:
    sumario = get_sumario(title=filme['titulo'], year=filme['ano']) # pegar sumario do filme com info básicas
    if(type(sumario) != str):                                       # recebeu um sumário com a requisição
        if('(' in filme['titulo']):                                 # verificar se tem () no título
            subtitulos = get_parenteses(filme['titulo'])            # testar requisição com subtitulos
            for subs in subtitulos:
                sumario = get_sumario(title=subs, year=filme['ano'])
                if(type(sumario) == str):
                    break
        elif (', ' in filme['titulo']):                             # verificar nome tá invertido
            sumario = get_sumario(title=set_artigo(filme['titulo']), year=filme['ano'])
    movies2.loc[(movies2.titulo==filme['titulo']) & (movies2.ano==filme['ano']),'sinopse'] = sumario    
    print(c,': ', filme['titulo'],': ', sumario)
    c+=1
    if(c%1000==0):
        movies2.to_csv('movies2.csv', index=False)

0 :  Mao Zedong 1949 :  
1 :  Sousse: Marché aux charbons (avec chameaux) :  
2 :  Nadide's Life :  nan
3 :  Screen Play :  nan
4 :  Vi på Saltkråkan :  nan
5 :  The World According to Xi Jinping :  
6 :  Mayflower, Quiltshoe & The Rubens Brothers :  nan
7 :  Ein Sommer in Salamanca :  
8 :  Rufus :  nan
9 :  Everything Is Terrible! Presents: The Great Satan :  nan
10 :  Обратная связь :  
11 :  Зайчонок и муха :  
12 :  6.5 Toman Per Meter :  nan
13 :  Gasht-e 2 :  nan
14 :  The American Experience: FDR :  nan
15 :  Didi e o Segredo dos Anjos :  
16 :  Snowboarďáci :  
17 :  The Good Soldier Svejk :  nan
18 :  Sun, Hay, Erotics :  nan
19 :  The Automatic Moving Company :  nan
20 :  Les grands moyens :  nan
21 :  Mail Order Wife :  nan
22 :  They Don't Wear Black Tie :  nan
23 :  nan :  Catherine Tate's iconic character Nan hits the big screen as she goes on a wild road trip from London to Ireland with her grandson Jamie to make amends with her estranged sister Nell. Militant vegan ars

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

# Lembrar de apagar a chave API ao divulgar o trabalho

In [56]:
get_sumario(prints=True)

Título: Star Wars
Sumário: Princess Leia is captured and held hostage by the evil Imperial forces in their effort to take over the galactic Empire. Venturesome Luke Skywalker and dashing captain Han Solo team together with the loveable robot duo R2-D2 and C-3PO to rescue the beautiful princess and restore peace and justice in the Empire.


'Princess Leia is captured and held hostage by the evil Imperial forces in their effort to take over the galactic Empire. Venturesome Luke Skywalker and dashing captain Han Solo team together with the loveable robot duo R2-D2 and C-3PO to rescue the beautiful princess and restore peace and justice in the Empire.'

In [77]:
pd.options.display.max_colwidth = 300
print(sinopses[sinopses.title=='Star Wars'].synopsis)

3    Princess Leia is captured and held hostage by the evil Imperial forces in their effort to take over the galactic Empire. Venturesome Luke Skywalker and dashing captain Han Solo team together with the loveable robot duo R2-D2 and C-3PO to rescue the beautiful princess and restore peace and justic...
Name: synopsis, dtype: object


In [50]:
movies2[(movies2.titulo=='Aladdin') & (movies2.ano==1992)]

,movieId,title,genres,titulo,ano,sinopse
580,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical,Aladdin,1992,Princess Jasmine grows tired of being forced t...
22286,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,Aladdin,1992,Princess Jasmine grows tired of being forced t...


In [45]:
get_sumario('Aladdin', 1992)

'Princess Jasmine grows tired of being forced to remain in the palace, so she sneaks out into the marketplace, in disguise, where she meets street-urchin Aladdin.  The couple falls in love, although Jasmine may only marry a prince.  After being thrown in jail, Aladdin becomes embroiled in a plot to find a mysterious lamp, with which the evil Jafar hopes to rule the land.'

In [49]:
movies2.loc[(movies2.titulo=='Aladdin') & (movies2.ano==1992),'sinopse'] = get_sumario('Aladdin', 1992)